In [1]:
import numpy as np
import pandas as pd

In [2]:
class DFP():
    def __init__(self, A, b, c, x0):
        self.A = A
        self.b = b
        self.x = x0
        self.c = c
        self.X, self.H, self.NormG, self.funValue = [], [], [], []
    # 定义二次函数
    def fun(self, x):
        return (x.T)@self.A@x/2 + self.b@x + self.c
    
    # 梯度
    def objGradient(self, x):
        g = self.A @ np.array(x) + self.b
        gNorm = np.linalg.norm(g, axis = 0)
        return g, gNorm
    
    # 迭代主体
    def iterate(self, epsilon):
        '''
        params:
        epsilon:迭代终止界限，梯度范数小于epsilon
        t0:每次区间探索的初始值
        h0:每次区间探索的初始探索步长
        alpha:加步系数
        '''
        self.X, self.H, self.NormG, self.funValue = [], [], [], []
        x = self.x
        self.X.append(x)
        g, gNorm = self.objGradient(x)
        self.NormG.append(gNorm)
        self.funValue.append(self.fun(x))# 记录迭代函数值
        while gNorm > epsilon:
            Hk, x, gNorm = self.Dfp(x, epsilon)
        if self.NormG == []:
            self.NormG.append(gNorm)
        self.H.append(Hk)
        return self.X, self.H, self.NormG, self.funValue
    
    def Dfp(self, initial_x, epsilon):
        gk, gNorm = self.objGradient(initial_x)
        xk = initial_x
        Hk = np.array([[1, 0], [0, 1]])
        pk = -Hk@gk
        for k in range(2):
            if gNorm < epsilon:
                return Hk, xk, gNorm
            else:
                self.H.append(Hk)
                xk_1 = xk + self.search_t(xk, pk)*pk
                # print(self.search_t(xk, Hk@pk))
                # print(xk_1)
                self.X.append(xk_1)
                self.funValue.append(self.fun(xk_1))
                gk_1, gNorm = self.objGradient(xk_1)
                Sk = xk_1 - xk
                yk = gk_1 - gk
                Hk = Hk + ((Sk@Sk.T)/(Sk.T@yk)) - (Hk@yk)@(yk.T@Hk)/(yk.T@Hk@yk)
                pk = -Hk@gk_1
                gk, xk = gk_1, xk_1
                self.NormG.append(gNorm)
        return Hk, xk_1, gNorm
    def search_t(self, x, direction):
        return -((self.b.T)@direction + direction.T@self.A@x/2 + x.T@self.A@direction/2)/(direction.T@self.A@direction)
    

In [3]:
A = np.array([[8, 0], [0, 2]])
b = np.array([-20, -12])
c = 136
epsilon = 0.01
x0 = np.array([8, 9])
sixth = DFP(A, b, c, x0)

In [4]:
X, H, NormG, funValue = sixth.iterate(epsilon)
res6 = pd.DataFrame({'迭代x值':X, 'H':H, '梯度范数':NormG, '函数值：':funValue})

In [5]:
res6

,迭代x值,H,梯度范数,函数值：
0,"[8, 9]","[[1, 0], [0, 1]]",44.407207,205.000000
1,"[2.4236503856041125, 8.239588688946016]","[[0.12673521850899738, -0.8732647814910026], [...",4.520631,80.039075
2,"[2.65023194319019, 8.177045602767302]","[[1, 0], [0, 1]]",4.516920,79.829806
3,"[2.154577826012214, 6.381386182104421]","[[0.4124073970660902, -0.5875926029339098], [-...",2.866719,75.622721
4,"[2.4910439432862197, 5.970653336713745]","[[1, 0], [0, 1]]",0.092620,75.001182
5,"[2.5038599892663456, 5.981152047197043]","[[0.17887400657599595, -0.8211259934240042], [...",0.048729,75.000415
6,"[2.4971521134703116, 5.987054908047133]","[[1, 0], [0, 1]]",0.034487,75.000200
7,"[2.502085077226393, 5.992660615012363]","[[0.2165186228742011, -0.7834813771257989], [-...",0.022220,75.000071
8,"[2.4989370192134492, 5.996045101846412]","[[1, 0], [0, 1]]",0.011614,75.000020
9,"[2.5005670857572118, 5.997561297446022]","[[0.19168579578138134, -0.8083142042186187], [...",0.006661,75.000007
